In [4]:
import numpy as np
import pandas as pd
import torch
import os
from cellpose import models, io
from PIL import Image
import torchvision.transforms as transforms

In [24]:
def load_camelyon17_dataset(dataset_path, metadata_path):
    """
    Carica il dataset Camelyon17 e i relativi metadati
    
    Args:
        dataset_path (str): Percorso della cartella contenente le patch
        metadata_path (str): Percorso del file di metadati
    
    Returns:
        DataFrame con informazioni delle patch
        Lista di percorsi delle patch
    """
    # Verifica l'esistenza dei percorsi
    if not os.path.exists(dataset_path):
        raise ValueError(f"Percorso dataset non esistente: {dataset_path}")
    if not os.path.exists(metadata_path):
        raise ValueError(f"Percorso metadata non esistente: {metadata_path}")
    
    # Carica i metadati
    metadata = pd.read_csv(metadata_path)
    
    # Lista per contenere tutti i percorsi delle patch
    patch_files = []
    
    # Attraversa ricorsivamente le sottocartelle
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.png'):
                # Aggiungi il percorso completo della patch
                patch_files.append(os.path.join(root, file))
    
    print(f"Trovate {len(patch_files)} patch")
    return metadata, patch_files

def extract_patch_metadata(patch_filepath):
    """
    Estrae i metadati dal nome della patch
    
    Args:
        patch_filepath (str): Percorso completo del file patch
    
    Returns:
        Dict con patient_id, node, x, y
    """
    # Estrai solo il nome del file dal percorso completo
    patch_filename = os.path.basename(patch_filepath)
    
    parts = patch_filename.replace('.png', '').split('_')
    return {
        'patient_id': parts[2],
        'node': parts[4],
        'x': int(parts[6]),
        'y': int(parts[8]),
        'filepath': patch_filepath  # Aggiungi il percorso completo
    }

In [25]:
def perform_cellpose_segmentation(patch_path):
    """
    Esegue la segmentazione con Cellpose
    
    Args:
        patch_path (str): Percorso della patch da processare
    
    Returns:
        Numero di cellule e densità cellulare
    """
    # Carica l'immagine con OpenCV
    image = cv2.imread(patch_path)
    
    if image is None:
        print(f"Errore: impossibile leggere l'immagine {patch_path}")
        return 0, 0
    
    # Converte l'immagine in scala di grigi se è a colori
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Carica il modello Cellpose preaddestrato
    model = models.Cellpose(gpu=False, model_type='cyto')
    
    try:
        # Esegui segmentazione
        masks, flows, styles = model.eval(image, diameter=8, channels=[0,0])
        
        # Calcola numero di cellule e densità
        num_cells = len(np.unique(masks)) - 1  # Sottrae lo sfondo
        cell_density = num_cells / (image.shape[0] * image.shape[1])
        
        return num_cells, cell_density
    
    except Exception as e:
        print(f"Errore durante la segmentazione di {patch_path}: {e}")
        return 0, 0

def update_metadata_with_cell_info(metadata, patch_files, dataset_path):
    """
    Aggiorna i metadati con informazioni cellulari
    """
    cell_data = []
    
    for patch_file in patch_files:
        try:
            # Estrai metadati del patch
            patch_meta = extract_patch_metadata(patch_file)
            
            # Esegui segmentazione
            num_cells, cell_density = perform_cellpose_segmentation(patch_file)
            
            cell_info = {
                **patch_meta,
                'num_cells': num_cells,
                'cell_density': cell_density
            }
            cell_data.append(cell_info)
        
        except Exception as e:
            print(f"Errore elaborazione patch {patch_file}: {e}")
    
    # Converti in DataFrame
    cell_metadata = pd.DataFrame(cell_data)
    
    return cell_metadata

In [26]:
def train_simclr(dataset_path, metadata_path):
    """
    Funzione principale per elaborare il dataset
    """
    # Carica il dataset
    metadata, patch_files = load_camelyon17_dataset(dataset_path, metadata_path)
    
    # Aggiorna metadati con informazioni cellulari
    updated_metadata = update_metadata_with_cell_info(metadata, patch_files, dataset_path)
    
    return updated_metadata

# Esempio di utilizzo
dataset_path = 'camelyon17_v1.0/patches/'
metadata_path = 'camelyon17_v1.0/metadata.csv'
metadata = train_simclr(dataset_path, metadata_path)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

